In [60]:
import pandas as pd
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import gensim

plt.style.use('seaborn')

#data_path = "data/santiago_covid_2020.csv"
data_path = "data/santiago_enero.csv"


# spanish stop words
stop_words = set(stopwords.words('spanish'))
stop_words = ['t', 'si', 'q', 'https', 'co', 'solo', 'ser', 'bien', 
            'así', 'ma', 'mas', 'igual', 'va', 'después',
            'hacer', 'hace', 'creo'] + list(stop_words)

# **Métodos útiles**

In [61]:
def preprocess_tweet(tweet):
    final = tweet.replace('#', '').replace(',', '').replace('.', '').replace('!', '').replace('?', '').replace('¿', '').split()
    final = [w.lower() for w in final]
    final = [w for w in final if w not in stop_words and len(w) > 3]
    return final

# **Tweets**

In [62]:
data = pd.read_csv(data_path)[['date', 'tweet']]
data.sample(5)

,date,tweet
10318,2021-01-29,00:59 en Cerrillos 10-2 BX3 Alaska y España h...
2717,2021-01-29,Vamo 🐷 #LuzesEternas #Palmeiras
8910,2021-01-29,"Acaba de publicar una foto en Talca,region del..."
13105,2021-01-01,16:25 en La Cisterna 10-3-1 RB5 Angamos y Almi...
871,2021-01-29,I think i have covid 😘


In [63]:
preprocess_tweet(data.iloc[0]['tweet'])

['will',
 'biden',
 'fauci',
 'take',
 'responsibility',
 'covid',
 'worse',
 'blame',
 'governors']

In [64]:
# Se procesan los tweets
processed_tweets = data['tweet'].map(preprocess_tweet)
processed_tweets.head()

0    [will, biden, fauci, take, responsibility, cov...
1          [through, another, year, covid, just, jump]
2    [@iamnickreynolds, @mattgaetz, think, about, t...
3    [covid, crisis, will, until, massive, fraud, c...
4    [@jkenney, planning, function, vaccine, covid,...
Name: tweet, dtype: object

In [65]:
# Se crea el vocabulario
dictionary = gensim.corpora.Dictionary(processed_tweets)

c = 0
for k, v in dictionary.iteritems():
    print(k,v)
    c += 1
    if c > 5: break
        
print('Largo del diccionario: ', len(dictionary))

0 biden
1 blame
2 covid
3 fauci
4 governors
5 responsibility
Largo del diccionario:  47081


In [66]:
# Se transforman las palabras a vectores con el dictionary
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_tweets]
bow_corpus[3636]

[(1757, 1), (2156, 1), (7554, 1), (14469, 1), (14470, 1)]

In [69]:
# Se crea y usa el lda model
# Running LDA using Bag of Words
lda_model = gensim.models.LdaMulticore(bow_corpus, \
                                    num_topics=5,\
                                    id2word=dictionary,\
                                    passes=2, workers=2)

In [70]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.011*"basura" + 0.009*"(10-2)" + 0.008*"fuego" + 0.005*"avenida" + 0.003*"jajajaja" + 0.003*"pastizales" + 0.003*"pasto" + 0.003*"santiago" + 0.003*"gracias" + 0.002*"2021"
Topic: 1 
Words: 0.018*"feliz" + 0.013*"nuevo" + 0.009*"gracias" + 0.008*"fuego" + 0.008*"psje" + 0.006*"humo" + 0.006*"estructura" + 0.005*"2021" + 0.005*"mejor" + 0.005*"llamado"
Topic: 2 
Words: 0.006*"feliz" + 0.005*"2020" + 0.004*"2021" + 0.004*"chile" + 0.004*"bueno" + 0.003*"quiero" + 0.003*"felicidades" + 0.003*"nuevo" + 0.002*"pedro" + 0.002*"covid"
Topic: 3 
Words: 0.008*"avenida" + 0.007*"chile" + 0.006*"santiago" + 0.006*"2021" + 0.006*"2020" + 0.005*"emanación" + 0.004*"(10-6)" + 0.004*"accidente" + 0.004*"colo" + 0.004*"vehicular"
Topic: 4 
Words: 0.024*"chile" + 0.020*"2021" + 0.018*"santiago" + 0.015*"feliz" + 0.012*"foto" + 0.011*"acaba" + 0.011*"publicar" + 0.007*"nuevo" + 0.005*"fuegos" + 0.005*"artificiales"
